## Segmenting LM Stack using segmented aligned em stack (e.g. from cellpose)

In cases where the quality of LM stack in z is not great to segment with high reliability,  using the carefully aligned em stack with e.g. BigWarp can be an option to use the segmentations.

**Preparation**:
  - Ensure the EM warped stack is aligned with the LM stack using BigWarp or a similar tool
  - Consider using a high-memory machine like vm1 from FAIM due to memory requirements
  - If encountering memory issues:
    - Use a high-memory machine (e.g., vm1 from FAIM)
    - Downsample the image stack
    - Crop the image to focus on specific regions of interest
    - Consider processing the stack in smaller chunks and stitching results (e.g. dask) 

**Cellpose Model Selection**:
  - Use the cyto3 model or a pretrained modified version

- **Segmentation Options**:
  - GUI: Use the Cellpose graphical user interface
  - Python: Write a custom Python script
  - CLI: Use the Cellpose command-line interface

- **Parameter Settings**:
  - Set flow threshold to 0
  - Set cell probability threshold to 1

- **Segmentation Process**:
  1. Load the EM warped stack into Cellpose
  2. Select the appropriate model (cyto3 or custom pretrained)
  3. Set the flow threshold to 0
  4. Set the cell probability threshold to 1
  5. Adjust other parameters as needed (e.g., cell diameter)
  6. Run the segmentation
  7. Review the results and adjust parameters if necessary



** for Cellpose CLI **
cellpose --dir /path/to/input/image --pretrained_model cyto3 --diameter 22 --flow_threshold 0 --cellprob_threshold 1 --do_3D --save_tif

** for python code **

In [ ]:
from cellpose import models
import tifffile

# Load your LM stack
image = tifffile.imread('/path/to/your/em_warped_stack.tif')

# Initialize model
model = models.Cellpose(model_type='cyto3', gpu=True)

# Set parameters
channels = [0, 0]  # grayscale image
diameter = 22    # estimated cell diameter

# Run segmentation
masks, flows, styles, diams = model.eval(image, 
                                         channels=channels,
                                         diameter=diameter,
                                         flow_threshold=0,
                                         cellprob_threshold=1,
                                         do_3D=True)

# Save the segmentation result
tifffile.imwrite('/path/to/output/segmentation.tif', masks)



**Validation**:
  - Visualize segmentation to LM stack 



In [ ]:
import napari
import tifffile

# Load original LM stack and segmentation
lm_stack = tifffile.imread('/path/to/your/lm_stack.tif')
segmentation = tifffile.imread('/path/to/output/segmentation.tif')

# Create Napari viewer
viewer = napari.Viewer()

# Add layers
viewer.add_image(lm_stack, name='LM Stack')
viewer.add_labels(segmentation, name='Cellpose Segmentation')

napari.run()